In [ ]:
import numpy as np
import nltk
from tqdm.notebook import tqdm
from glob import glob
import fasttext
from navec import Navec

from transformers import AutoTokenizer, AutoModel, BertTokenizer, BertModel
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset, DataLoader
import torch.functional as F
from torch import nn
import torchmetrics
import pytorch_lightning as pl

from warnings import filterwarnings
filterwarnings("ignore")

## Data

In [ ]:
from model import CustomDataset

In [ ]:
# embeddings
navec_model = Navec.load("data/navec_hudlit_v1_12B_500K_300d_100q.tar")
fasttext_model = fasttext.load_model("data/cc.ru.300.bin")

# Load data
train_data = glob("data/augmentations/train/*.npy")
val_data = glob("data/augmentations/val/*.npy")

len(train_data), len(val_data)

In [ ]:
sent_size = 112
batch_size = 128

# dataloaders
dataset_train = CustomDataset(train_data, sent_size, True, navec_model, fasttext_model)
dataloader_train = DataLoader(dataset_train, batch_size=batch_size, shuffle=True)

dataset_val = CustomDataset(val_data, sent_size, True, navec_model, fasttext_model)
dataloader_val = DataLoader(dataset_val, batch_size=batch_size, shuffle=True)

In [ ]:
for x in dataloader_train:
    break

## Model

In [ ]:
from model import FinalModel
import optuna

In [ ]:
def objective(trial: optuna.trial.Trial):

    # params
    lr = trial.suggest_float("lr", 2e-5, 2e-3)
    weight_decay = trial.suggest_float("weight_decay", 1e-8, 1e-1)
    hidden_size = trial.suggest_int("hidden_size", 4, 256)
    bidirectional = trial.suggest_categorical("bidirectional", [True, False])
    drop_lstm = trial.suggest_float("drop_lstm", 0.0, 0.6)
    drop_linear = trial.suggest_float("drop_linear", 0.0, 0.6)
    linear1_size = trial.suggest_int("linear1_size", 256, 1024)
    linear2_size = trial.suggest_int("linear2_size", 256, 1024)

    
    params = {"lr": lr, "weight_decay": weight_decay, 
              "hidden_size": hidden_size,  "bidirectional": bidirectional,
              "drop_lstm":drop_lstm, "drop_linear":drop_linear,
              "linear1_size":linear1_size, "linear2_size":linear2_size}
    
    # model
    model = FinalModel(**params)
    
    # model utils
    lr_monitoring = pl.callbacks.LearningRateMonitor(logging_interval="epoch")
    early_stop_callback = pl.callbacks.early_stopping.EarlyStopping(monitor="val_f1", min_delta=0.0001,
                                                                    patience=3, verbose=False, mode="max")
    logger = pl.loggers.TensorBoardLogger(save_dir="logs", name="optuna")
    
    # train
    trainer = pl.Trainer(gpus=1, max_epochs=15, logger=logger,
                         callbacks=[lr_monitoring, early_stop_callback], weights_summary=None)
    trainer.logger.log_hyperparams(params)
    trainer.fit(model, dataloader_train, dataloader_val)
    
    return trainer.callback_metrics["val_f1"].item()

In [ ]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=30, timeout=4*60*60)